In [39]:
import pandas as pd

In [40]:
df = pd.read_csv('../MNH TTO Data/MNH TTO Data - MERGED.csv')

In [41]:
df

,Date,Bed no.,Case no.,Can close,No. of drugs,Time TTO received (24 hr format) (eg 1340),Timing of Packing (24 hr format / PBH) (eg 1340 / PBH),Timing of Dispensing (24 hr format),Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),...,Timing of Dispensing,20 min and below,Between 20 to 30 min,30min and below,More than 30 min,Weekday,Weekday Count,Weekend,Weekend Count,NOT Ward 9
0,01-01-21,905,4020095917,839,4,828,pbh,840,NaN,PBH,...,8:40 AM,1,0,1,0,TRUE,1,FALSE,0,0
1,01-01-21,1202,4020097017,934,8,904,850,NaN,wait for 2nd rx,##############################################...,...,#VALUE!,0,0,0,0,TRUE,1,FALSE,0,1
2,01-01-21,1224,4020095632,934,1,901,905,920,NaN,0:04,...,9:20 AM,1,0,1,0,TRUE,1,FALSE,0,1
3,01-01-21,818,4020097033,934,1,900,901,915,NaN,0:01,...,9:15 AM,1,0,1,0,TRUE,1,FALSE,0,1
4,01-01-21,803,4020097026,1029,1,908,915,1027,"can close, added on metformin and glipizide. 1...",0:07,...,10:27 AM,0,0,0,1,TRUE,1,FALSE,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15517,31-12-21,824,4021107656,1311,1,1300,1300,1310,add on TTO,0:00,...,1:10 PM,1,0,1,0,TRUE,1,FALSE,0,1
15518,31-12-21,1205,4021107518,1351,1,1335,pbh,1347,NaN,PBH,...,1:47 PM,1,0,1,0,TRUE,1,FALSE,0,1
15519,31-12-21,1131,4021107644,1426,3,1408,1417,1434,NaN,0:09,...,2:34 PM,0,1,1,0,TRUE,1,FALSE,0,1
15520,31-12-21,622,4021108689,1446,2,1359,1409,1441,1431 pt in bathroom,0:10,...,2:41 PM,0,0,0,1,TRUE,1,FALSE,0,1


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15522 entries, 0 to 15521
Data columns (total 23 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   Date                                                    15522 non-null  object
 1   Bed no.                                                 15512 non-null  object
 2   Case no.                                                15516 non-null  object
 3   Can close                                               15455 non-null  object
 4   No. of drugs                                            15520 non-null  object
 5   Time TTO received (24 hr format) (eg 1340)              15518 non-null  object
 6   Timing of Packing (24 hr format / PBH) (eg 1340 / PBH)  15509 non-null  object
 7   Timing of Dispensing (24 hr format)                     15512 non-null  object
 8   Comments (for delayed cases) (eg. Pt sleeping 

In [43]:
df.isnull().count()

Date                                                      15522
Bed no.                                                   15522
Case no.                                                  15522
Can close                                                 15522
No. of drugs                                              15522
Time TTO received (24 hr format) (eg 1340)                15522
Timing of Packing (24 hr format / PBH) (eg 1340 / PBH)    15522
Timing of Dispensing (24 hr format)                       15522
Comments (for delayed cases) (eg. Pt sleeping etc)        15522
Time Taken To Pack (mins)                                 15522
Time Taken to Reach Pt (mins)                             15522
Time TTO received                                         15522
Timing of packing                                         15522
Timing of Dispensing                                      15522
20 min and below                                          15522
Between 20 to 30 min                    

In [44]:
df.drop(['Can close'], axis=1, inplace=True)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15522 entries, 0 to 15521
Data columns (total 22 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   Date                                                    15522 non-null  object
 1   Bed no.                                                 15512 non-null  object
 2   Case no.                                                15516 non-null  object
 3   No. of drugs                                            15520 non-null  object
 4   Time TTO received (24 hr format) (eg 1340)              15518 non-null  object
 5   Timing of Packing (24 hr format / PBH) (eg 1340 / PBH)  15509 non-null  object
 6   Timing of Dispensing (24 hr format)                     15512 non-null  object
 7   Comments (for delayed cases) (eg. Pt sleeping etc)      3862 non-null   object
 8   Time Taken To Pack (mins)                     

In [46]:
df.drop(df.iloc[:, 13:22], inplace = True, axis = 1)

In [48]:
df.describe()

,Date,Bed no.,Case no.,No. of drugs,Time TTO received (24 hr format) (eg 1340),Timing of Packing (24 hr format / PBH) (eg 1340 / PBH),Timing of Dispensing (24 hr format),Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO received,Timing of packing,Timing of Dispensing
count,15522,15512,15516,15520,15518,15509,15512,3862,15521,15522,15522,15522,15522
unique,392,403,14887,30,876,843,786,2339,182,343,870,821,782
top,12-03-21,626,4021052393,1,900,pbh,930,paeds relabel,PBH,0:25,9:00 AM,PBH,9:30 AM
freq,108,120,4,3653,86,3795,192,163,3843,395,86,3853,192


In [49]:
df

,Date,Bed no.,Case no.,No. of drugs,Time TTO received (24 hr format) (eg 1340),Timing of Packing (24 hr format / PBH) (eg 1340 / PBH),Timing of Dispensing (24 hr format),Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO received,Timing of packing,Timing of Dispensing
0,01-01-21,905,4020095917,4,828,pbh,840,NaN,PBH,0:12,8:28 AM,PBH,8:40 AM
1,01-01-21,1202,4020097017,8,904,850,NaN,wait for 2nd rx,##############################################...,blank,9:04 AM,8:50 AM,#VALUE!
2,01-01-21,1224,4020095632,1,901,905,920,NaN,0:04,0:19,9:01 AM,9:05 AM,9:20 AM
3,01-01-21,818,4020097033,1,900,901,915,NaN,0:01,0:15,9:00 AM,9:01 AM,9:15 AM
4,01-01-21,803,4020097026,1,908,915,1027,"can close, added on metformin and glipizide. 1...",0:07,1:19,9:08 AM,9:15 AM,10:27 AM
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15517,31-12-21,824,4021107656,1,1300,1300,1310,add on TTO,0:00,0:10,1:00 PM,1:00 PM,1:10 PM
15518,31-12-21,1205,4021107518,1,1335,pbh,1347,NaN,PBH,0:12,1:35 PM,PBH,1:47 PM
15519,31-12-21,1131,4021107644,3,1408,1417,1434,NaN,0:09,0:26,2:08 PM,2:17 PM,2:34 PM
15520,31-12-21,622,4021108689,2,1359,1409,1441,1431 pt in bathroom,0:10,0:42,1:59 PM,2:09 PM,2:41 PM
